# 安裝相關套件

In [ ]:
!sudo apt-get install libxml2-dev libxslt1-dev python-dev
# install html parser
!sudo apt-get install python-lxml
# install chinease cuting package
!sudo pip install jieba
# install beautifulsoup4   
!sudo pip install BeautifulSoup4

# 欲抓的網頁列表

In [ ]:
urllist = ['http://lohas.pixnet.net/blog/post/31969271',
           'http://lohas.pixnet.net/blog/post/32286983',
           'http://lohas.pixnet.net/blog/post/31423623',
           'http://lohas.pixnet.net/blog/post/31466711',
           'http://lohas.pixnet.net/blog/post/40148737',
          ]

# 下載網頁並且組成一個檔案

In [ ]:
import urllib2
import json
f = open('./pixnet.txt',"w")

for u in urllist:
    line = {}
    response = urllib2.urlopen(u)
    html = response.read()
    html = html.replace('\r','').replace('\n','')
    line['html'] = html
    line['url'] =u
    
    line_str = json.dumps(line)
    
    f.write(line_str+"\r\n")
f.close()

# 我們來看一下實際檔案的內容
[請點即此鏈結](http://localhost:8001/edit/spark_tutorial/pixnet.txt)

# 載入頁面，並進行 json parsing

In [ ]:
pixnet = sc.textFile('./pixnet.txt',use_unicode=False).map(
    lambda x : json.loads(x)).map(lambda x : (x['url'],x['html']))

print "URL:", pixnet.first()[0]
print
print "HTML:", pixnet.first()[1][:200]


## RDD 常用的 Function 如下
* map(funct) - 對 RDD 的裡頭的元素進行處理動作
* mapValues (func) - 排除 Key,只對 RDD 的裡頭的元素進行處理動作
* reduceByKey(func) - 將相同的 Key 裡頭的 Values 給予合併起來
* count() - 計算 RDD 裡頭元素的個數
* filter(func) - 根據 condition 判斷是否需要保留
* first() - 取得 RDD 裡頭的第一個元素

## <span style="color: blue"> 請填入??，來計算"好吃"的次數(多少頁面之中)</span>

In [ ]:
count_nummber = pixnet.filter(lambda x : u"好吃" in x[1] ).count()
if count_nummber == 5 : print "你答對了"

##<span style="color: blue">請修改以下的程式碼，並計算"很好"所有出現次數，注意!!不是頁面數</span>
 <span style="color:red">提示:修改 word_count 函式</span>

In [ ]:
def word_count(text):
    return text.count(u"好吃")

print "很好出現了",word_count(u"老師很好很好很好很好！！！！"),"次"

In [ ]:
pixnet.mapValues(word_count).map(lambda x : x[1]).sum()

# Use Beautiful Soaup Parse HTML

In [ ]:
mydivs = None
def analysis_content(content):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(content)
    mydivs = soup.findAll("div", { "class" : "article-content-inner" })
    return mydivs[0].getText()


In [ ]:
print pixnet.map(analysis_content).first()[:100]

In [ ]:
sc.parallelize(urllist)